----

# **Stacking Ensamble Learning Techinque**

## **Author**   :  **Muhammad Adil Naeem**

## **Contact**   :   **madilnaeem0@gmail.com**
<br>

----



### **Import Libraries**

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from vecstack import stacking

import warnings
warnings.filterwarnings('ignore')

### **Load Dataset**

In [26]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
names = ['Class', 'Alcohol', 'Malic acid', 'Ash',
         'Alcalinity of ash', 'Magnesium', 'Total phenols',
         'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
         'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
         'Proline']

df = pd.read_csv(link, header=None, names=names)
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


### **Detailed Information about Dataset**

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Class                         178 non-null    int64  
 1   Alcohol                       178 non-null    float64
 2   Malic acid                    178 non-null    float64
 3   Ash                           178 non-null    float64
 4   Alcalinity of ash             178 non-null    float64
 5   Magnesium                     178 non-null    int64  
 6   Total phenols                 178 non-null    float64
 7   Flavanoids                    178 non-null    float64
 8   Nonflavanoid phenols          178 non-null    float64
 9   Proanthocyanins               178 non-null    float64
 10  Color intensity               178 non-null    float64
 11  Hue                           178 non-null    float64
 12  OD280/OD315 of diluted wines  178 non-null    float64
 13  Proli

### **Splitting Data into Dependent and Independent Variables**

In [28]:
X = df.iloc[:,1:]
y = df[['Class']]

### **Splitting Data into Train and Test**

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### **Ensemble of Classification Models**

- This code defines a list of three classification models: K-Nearest Neighbors (KNN), Random Forest, and XGBoost, each configured with specific parameters for use in an ensemble learning approach.

In [30]:
models = [
    KNeighborsClassifier(n_neighbors=5,n_jobs=-1),
    RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=5),
    XGBClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=5),

]

### **Stacking Classifiers with Out-of-Fold Predictions**

- This code uses a stacking technique to combine multiple classifiers from the `models` list. It trains the models on `X_train` with `y_train`, generates out-of-fold predictions, and evaluates their performance on `X_test`.
- The stacking process employs cross-validation with 4 folds, stratification, and shuffling, while reporting progress with verbosity.

In [31]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

S_train,S_test  = stacking(models, X_train, y_train,
                           X_test, regression=False, mode='oof_pred_bag',needs_proba=False,
                           save_dir=None,metric=accuracy_score, n_folds=4, stratified=True,
                           shuffle=True, random_state=0, verbose=2)

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.69444444]
    fold  1:  [0.63888889]
    fold  2:  [0.62857143]
    fold  3:  [0.65714286]
    ----
    MEAN:     [0.65476190] + [0.02509117]
    FULL:     [0.65492958]

model  1:     [RandomForestClassifier]
    fold  0:  [0.97222222]
    fold  1:  [0.97222222]
    fold  2:  [1.00000000]
    fold  3:  [1.00000000]
    ----
    MEAN:     [0.98611111] + [0.01388889]
    FULL:     [0.98591549]

model  2:     [XGBClassifier]
    fold  0:  [0.94444444]
    fold  1:  [0.91666667]
    fold  2:  [0.97142857]
    fold  3:  [0.97142857]
    ----
    MEAN:     [0.95099206] + [0.02267381]
    FULL:     [0.95070423]

